In [ ]:
import fcm
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import numpy as np
import pandas as pd
import seaborn as sns
import gtools as gt
from gtools import crater_tools

## Example clusters

In [ ]:
synthetics=pd.read_csv('./model-data/output-198.csv')
clusters = pd.read_csv('./crater-data/craters-198.csv', index_col=[0,1])

#ESP_037706_1765    
fig = plt.figure(figsize=(7.5,5))
plt.subplot(2,3,1)
img = pimg.imread('./Figures/ESP_037706_1765.png')
plt.imshow(img)
plt.annotate('(a)', xy=(0.05, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
plt.axis('off')

ax = plt.subplot(2,3,2)
cluster = pd.read_excel("./obs-data/ESP_037706_1765.xlsx", usecols=[1,2,3], names=['lon', 'lat', 'r'])
cluster['r'] = cluster['r']*0.5
crater_tools.lon_lat_to_meters(cluster, Rp=3390000.)
c = crater_tools.cluster_characteristics(cluster)
crater_tools.plot_craters(cluster, ellipse_color='blue', 
                          px_lim=[-60,60], py_lim=[-60,60], fig=fig, ax=ax, flipxy=True)
plt.annotate('(b)', xy=(0.05, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
plt.tick_params(labelsize=8)
for info in c:
    print(info, c[info])

# Plot the similar simulation (ID 5442)
ax = plt.subplot(2,3,3)
for ID, cluster in clusters.groupby(level=0):
    if ID == 5442:
        crater_tools.plot_craters(cluster, ellipse_color='blue', 
                                  px_lim=[-60,60], py_lim=[-60,60], fig=fig, ax=ax)
        plt.tick_params(labelsize=8)
        c = crater_tools.cluster_characteristics(cluster)
        for info in c:
            print(info, c[info])
plt.annotate('(c)', xy=(0.05, 0.9), ha='left', xycoords='axes fraction', fontsize=10)


#ESP_038458_2030
plt.subplot(2,3,4)
img = pimg.imread('./Figures/ESP_038458_2030.png')
plt.imshow(img)
plt.annotate('(d)', xy=(0.05, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
plt.axis('off')

ax = plt.subplot(2,3,5)
cluster = pd.read_excel("./obs-data/ESP_038458_2030.xlsx", usecols=[1,2,3], names=['lon', 'lat', 'r'])
cluster['r'] = cluster['r']*0.5
crater_tools.lon_lat_to_meters(cluster, Rp=3390000.)
c = crater_tools.cluster_characteristics(cluster)
obs = crater_tools.plot_craters(cluster, ellipse_color='blue', 
                                px_lim=[-200,200], py_lim=[-200,200], fig=fig, ax=ax, flipxy=True)
plt.annotate('(e)', xy=(0.05, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
plt.tick_params(labelsize=8)
for info in c:
    print(info, c[info])
    
# Plot the simulation (ID 8029)
ax = plt.subplot(2,3,6)
for ID, cluster in clusters.groupby(level=0):
    if ID == 8029:
        mod = crater_tools.plot_craters(cluster, ellipse_color='blue', 
                                        px_lim=[-200,200], py_lim=[-200,200], fig=fig, ax=ax)
        plt.tick_params(labelsize=8)
        c = crater_tools.cluster_characteristics(cluster)
        for info in c:
            print(info, c[info])
plt.annotate('(f)', xy=(0.05, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
            
plt.tight_layout()
plt.savefig('./Figures/Figure1.pdf')

## Best-fit model - cumulative distributions

In [ ]:
#### Comparison between Model and Observation
ID = "198"
Deff_c = 5.65; N_c = 5
synthetics=pd.read_csv('./model-data/output-'+ID+'.csv') #.sample(2092, replace=True)

# Get column headings from Matt's file
observedclusters=pd.read_csv('./obs-data/observedclusters.csv')

# Fields to plot, limits and labels. . .
characteristics = ["Effective Diameter [m]", "No. of Craters", "Dispersion [m]", "Aspect Ratio",
                   "Maximum Diameter [m]", "No. of Large Craters", "Large Crater Fraction", "CSFD exponent"]
xlimits = [[2., 100.], [5, 1000.], [3.E0, 3.E3], [0., 1.], 
           [2., 100.], [1, 100.], [0., 1.], [-3.5, -0.]]
lab = ["(a)", "(b)", "(c)", "(d)", "(e)", "(f)", "(g)", "(h)"]

observedsingulars=pd.read_csv('./obs-data/observedsingulars.csv')
observed = pd.concat([observedclusters, observedsingulars], sort=False)

syntheticclusters, syntheticsingulars, syntheticbursts = gt.classify_clusters(synthetics)
print()
print('Observed: ')
gt.cluster_stats(observedclusters, observedsingulars)
print()
print('Model: ')
gt.cluster_stats(syntheticclusters, syntheticsingulars)
print()

ss_all = syntheticsingulars.copy()
os_all = observedsingulars.copy()
sc_all = syntheticclusters.copy()
oc_all = observedclusters.copy()

scs = []
nsamp = int(len(syntheticclusters)*200 /
            len(syntheticclusters[syntheticclusters['Effective Diameter [m]'] > 10.]))
for i in range(40):
    sc = syntheticclusters.sample(nsamp, replace=False)
    scs.append(sc[(sc['No. of Craters'] > N_c) & (sc['Effective Diameter [m]'] > Deff_c)])

# Filter the results and data to exclude small clusters and those with few craters
syntheticclusters = syntheticclusters[syntheticclusters['No. of Craters'] > N_c]
syntheticclusters = syntheticclusters[syntheticclusters['Effective Diameter [m]'] > Deff_c]
observedclusters = observedclusters[observedclusters['No. of Craters'] > N_c]
observedclusters = observedclusters[observedclusters['Effective Diameter [m]'] > Deff_c]

# Print out the D>10 stats
for size in [5.65, 10.]:
    try:
        print('Observed D > ', size, len(observedclusters[observedclusters['Effective Diameter [m]'] > size]),
              len(observedsingulars[observedsingulars['Effective Diameter [m]'] > size]),
              len(observedclusters[observedclusters['Effective Diameter [m]'] > size])/
              len(observedsingulars[observedsingulars['Effective Diameter [m]'] > size]))
        print('Synthetic D > ', size, len(syntheticclusters[syntheticclusters['Effective Diameter [m]'] > size]), 
              len(syntheticsingulars[syntheticsingulars['Effective Diameter [m]'] > size]), 
              len(syntheticclusters[syntheticclusters['Effective Diameter [m]'] > size]) / 
              len(syntheticsingulars[syntheticsingulars['Effective Diameter [m]'] > size]), len(syntheticbursts))
    except ZeroDivisionError:
        print('No singulars so not reporting stats.')
    
print()

fig =plt.figure(figsize=(7.5,5))
ax=[fig.add_subplot(2,4,i) for i in range(1,9)]
for char,a in zip(characteristics,range(9)):

    # Plot observations
    ocdf = observedclusters[char].value_counts().sort_index()[::-1].cumsum()/len(observedclusters[char])
    ocdf.plot(label='Obs.',color='Black', linewidth=2, alpha=0.5, ax=ax[a])

    # Plot model results
    cdf = syntheticclusters[char].value_counts().sort_index()[::-1].cumsum()/len(syntheticclusters[char])
    cdf.plot(label='Model', linewidth=1, ax=ax[a])

    # Uncertainty
    for sc in scs:
        cdf = sc[char].value_counts().sort_index()[::-1].cumsum()/len(sc[char])
        cdf.plot(label='', color='b', linewidth=1, ax=ax[a], alpha=0.1)
        
    # Labels, limits, etc.
    ax[a].set_title(lab[a], fontsize=8);
    ax[a].set_xlabel(char, fontsize=8)
    if (char != "CSFD exponent" and char != "Aspect Ratio" and 
        char != "Aspect Ratio" and char != "Large Crater Fraction"):
        ax[a].set_xscale('log')
        ax[a].set_ylim(0., 1.E0)
    else:
        ax[a].set_ylim(0., 1.)
    ax[a].set_xlim(xlimits[a])
    ax[a].legend(fontsize=8)
    ax[a].tick_params(labelsize=8)

ax[0].set_ylabel('Cumulative Frequency', fontsize=8)
plt.tight_layout()
plt.show()
fig.savefig('./Figures/Figure3.pdf')
plt.close()

## Histograms of crater size and number

In [ ]:
# sqrt(2) bins from 1-100 m
bin_list = np.logspace(0.,1.9567,14)
var = 'Effective Diameter [m]'

# Generate counts for synthetic data (so that they can be scaled if necessary)
counts_m, bins_m = np.histogram(sc_all[var], bins=bin_list)
counts_m = counts_m*0.2  # Scale to same size as observational dataset

# Or, consider variation by subsampling
mcounts = []
for i in range(len(counts_m)):
    mcounts.append([counts_m[i]])
for i in range(50):
    to = sc_all.sample(int(len(sc_all)/5), replace=True)
    counts, _ = np.histogram(to[var], bins=bin_list)
    for i in range(len(mcounts)):
        mcounts[i].append(counts[i])
errors = []
for i in range(len(mcounts)):
    errors.append(np.std(mcounts[i]))
    
# Generate counts for observational data
counts_o, bins_o = np.histogram(oc_all[var], bins=bin_list)

# Measure uncertainty using simple Poisson statistics
errors_o = np.sqrt(counts_o)

# Plot the data
fig =plt.figure(figsize=(3.75,7.5))
plt.subplot(3,1,2)
plt.errorbar(bins_m[:-1]*2**0.25, counts_m, errors, ls='', fmt='s', 
             color=sns.color_palette("Paired")[0], label='Model')
plt.errorbar(bins_m[:-1]*2**0.25, counts_o, errors_o, ls='', fmt='o', 
             color=sns.color_palette("Paired")[1], label='Observed')
plt.legend(fontsize=8)
plt.xlabel(var, fontsize=8)
plt.xscale('log')
plt.ylabel('Frequency', fontsize=8)
plt.yscale('log')
plt.ylim(0.1, 400)
plt.axvline(5.65, color='k', ls='dotted')
plt.tick_params(labelsize=8)
plt.annotate('(b)', xy=(0.1, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

# Generate counts for synthetic data (so that they can be scaled if necessary)
counts_m, bins_m = np.histogram(ss_all[var], bins=bin_list)
counts_m = counts_m*0.2  # Scale to same size as observational dataset

# Or, consider variation by subsampling
mcounts = []
for i in range(len(counts_m)):
    mcounts.append([counts_m[i]])
for i in range(50):
    to = ss_all.sample(int(len(ss_all)/5), replace=True)
    counts, _ = np.histogram(to[var], bins=bin_list)
    for i in range(len(mcounts)):
        mcounts[i].append(counts[i])
errors = []
for i in range(len(mcounts)):
    errors.append(np.std(mcounts[i]))
    
# Generate counts for observational data
counts_o, bins_o = np.histogram(os_all[var], bins=bin_list)

# Measure uncertainty using simple Poisson statistics
errors_o = np.sqrt(counts_o)

# Plot the data
plt.subplot(3,1,1)
plt.errorbar(bins_m[:-1]*2**0.25, counts_m, errors, ls='', fmt='s', 
             color=sns.color_palette("Paired")[0], label='Model')
plt.errorbar(bins_m[:-1]*2**0.25, counts_o, errors_o, ls='', fmt='o', 
             color=sns.color_palette("Paired")[1], label='Observed')
plt.legend(fontsize=8)
plt.xlabel(var, fontsize=8)
plt.xscale('log')
plt.ylabel('Frequency', fontsize=8)
plt.ylim(0.1, 400)
plt.yscale('log')
plt.tick_params(labelsize=8)
plt.axvline(5.65, color='k', ls='dotted')
plt.annotate('(a)', xy=(0.1, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

sc_lar = sc_all[sc_all['Effective Diameter [m]'] > Deff_c]
oc_lar = oc_all[oc_all['Effective Diameter [m]'] > Deff_c]

bin_list = np.logspace(np.log10(2.),np.log10(16384),14)
var = 'No. of Craters'

# Generate counts for synthetic data (so that they can be scaled if necessary)
counts_m, bins_m = np.histogram(sc_lar[var], bins=bin_list)
counts_m = counts_m*0.2  # Scale to same size as observational dataset

# Or, consider variation by subsampling
mcounts = []
for i in range(len(counts_m)):
    mcounts.append([counts_m[i]])
for i in range(50):
    to = sc_lar.sample(int(len(sc_lar)/5), replace=True)
    counts, _ = np.histogram(to[var], bins=bins_m)
    for i in range(len(mcounts)):
        mcounts[i].append(counts[i])
#print(mcounts) 
errors = []
for i in range(len(mcounts)):
    errors.append(np.std(mcounts[i]))
    
# Generate counts for observational data
counts_o, bins_o = np.histogram(oc_lar[var], bins=bins_m)

# Measure uncertainty using simple Poisson statistics
errors_o = np.sqrt(counts_o)

# Plot the data
plt.subplot(3,1,3)
plt.errorbar(bins_m[:-1]*2**0.5, counts_m, errors, ls='', fmt='s', 
             color=sns.color_palette("Paired")[0], label='Model')
plt.errorbar(bins_m[:-1]*2**0.5, counts_o, errors_o, ls='', fmt='o', 
             color=sns.color_palette("Paired")[1], label='Observed')
plt.annotate('(c)', xy=(0.1, 0.9), ha='right', xycoords='axes fraction', fontsize=10)
plt.legend(fontsize=8)
plt.xlabel(var, fontsize=8)
plt.axvline(5, color='k', ls='dotted')
plt.xscale('log')
plt.ylabel('Frequency', fontsize=8)
plt.yscale('log')
plt.ylim(0.1, 400)
plt.tick_params(labelsize=8)
plt.tight_layout()
plt.savefig('./Figures/Figure2.pdf')

## KDE Plots comparing distributions

In [ ]:
logobserved = observedclusters.copy()
logobserved['No. of Craters'] = np.log10(logobserved['No. of Craters'])
logobserved['No. of Large Craters'] = np.log10(logobserved['No. of Large Craters'])
logobserved['Dispersion [m]'] = np.log10(logobserved['Dispersion [m]'])
logobserved.rename(columns={"Dispersion [m]": "log(Dispersion [m])", 
                            "No. of Craters": "log(No. of Craters)",
                            "No. of Large Craters": "log(No. of Large Craters)"}, inplace=True)

logmodel = syntheticclusters.copy()
logmodel['No. of Craters'] = np.log10(logmodel['No. of Craters'])
logmodel['No. of Large Craters'] = np.log10(logmodel['No. of Large Craters'])
logmodel['Dispersion [m]'] = np.log10(logmodel['Dispersion [m]'])
logmodel.rename(columns={"Dispersion [m]": "log(Dispersion [m])", 
                         "No. of Craters": "log(No. of Craters)",
                         "No. of Large Craters": "log(No. of Large Craters)"}, inplace=True)

In [ ]:
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8}) 
sns.color_palette("light:b", as_cmap=True)
fig = plt.figure(figsize=(7.5,10))

fig.add_subplot(4,3,1)
var1 = 'Effective Diameter [m]'
var2 = 'log(No. of Craters)'
xlimit = [0., 35.]
ylimit=[0.,3.]
plt.title('Observations', fontsize=8)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
plt.annotate('(a)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)
plt.xlim(xlimit)
plt.ylim(ylimit)
fig.add_subplot(4,3,2)
plt.title('Model')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=True, cmap='light:b')
plt.annotate('(b)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)
plt.xlim(xlimit)
plt.ylim(ylimit)
fig.add_subplot(4,3,3)
plt.title('Comparison', fontsize=8)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=False)
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(c)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,4)
var1 = 'Effective Diameter [m]'
var2 = 'log(Dispersion [m])'
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
xlimit = [0., 35.]
ylimit=[0.5,3.]
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(d)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,5)
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=True, cmap='light:b')
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(e)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,6)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=False)
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(f)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,7)
var1 = 'Effective Diameter [m]'
var2 = 'Aspect Ratio'
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
xlimit = [0.,35.]
ylimit=[0.,1.]
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(g)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,8)
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=True, cmap='light:b')
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(h)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,9)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=False)
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(i)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,10)
var1 = 'Effective Diameter [m]'
var2 = 'Large Crater Fraction'
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
xlimit = [0.,35.]
#ylimit=[0.,1.]
plt.xlim(xlimit)
#plt.ylim(ylimit)
plt.annotate('(j)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,11)
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=True, cmap='light:b')
plt.xlim(xlimit)
#plt.ylim(ylimit)
plt.annotate('(k)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,12)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=False)
plt.xlim(xlimit)
#plt.ylim(ylimit)
plt.annotate('(l)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.tight_layout()
fig.savefig('Figures/Figure4.pdf')

In [ ]:
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8}) 
sns.color_palette("light:b", as_cmap=True)
fig = plt.figure(figsize=(7.5,10))

fig.add_subplot(4,3,1)
var1 = 'log(No. of Craters)'
var2 = 'log(Dispersion [m])'
xlimit = [0., 3.]
ylimit = [0.5, 3.]
plt.title('Observations', fontsize=8)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
plt.annotate('(a)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)
plt.xlim(xlimit)
plt.ylim(ylimit)
fig.add_subplot(4,3,2)
plt.title('Model')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=True, cmap='light:b')
plt.annotate('(b)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)
plt.xlim(xlimit)
plt.ylim(ylimit)
fig.add_subplot(4,3,3)
plt.title('Comparison', fontsize=8)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=False)
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(c)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,4)
var1 = 'log(No. of Craters)'
var2 = 'Aspect Ratio'
xlimit = [0.,3.]
ylimit = [0.,1.]
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(d)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,5)
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=True, cmap='light:b')
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(e)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,6)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=False)
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(f)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,7)
var1 = 'Aspect Ratio'
var2 = 'log(Dispersion [m])'
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
xlimit = [0.,1.]
ylimit = [0.5, 3.]
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(g)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,8)
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=True, cmap='light:b')
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(h)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,9)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=False)
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(i)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,10)
var1 = 'log(No. of Craters)'
var2 = 'CSFD exponent'
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
xlimit = [0., 3.]
ylimit=[-3.5,0.]
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(j)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,11)
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=True, cmap='light:b')
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(k)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.add_subplot(4,3,12)
sns.kdeplot(x=logobserved[var1], y=logobserved[var2], shade=True, cmap='light:b')
sns.kdeplot(x=logmodel[var1], y=logmodel[var2], shade=False)
plt.xlim(xlimit)
plt.ylim(ylimit)
plt.annotate('(l)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

fig.tight_layout()
fig.savefig('Figures/Figure5.pdf')

## Convergence of results

In [ ]:
### Let's determine how many samples are required for a robust score. . .
Deff_c = 10.; N_c = 5

s = pd.concat([pd.read_csv("./model-data/output-196.csv"), 
               pd.read_csv("./model-data/output-197.csv"), 
               pd.read_csv("./model-data/output-198.csv")])
#s = pd.read_csv("./model-data/output-198.csv")
sc, ss, sb = gt.classify_clusters(s)
gt.cluster_stats(sc, ss)

# Filter the results and data to exclude small clusters and those with few craters
sc = sc[(sc['No. of Craters'] > N_c) & (sc['Effective Diameter [m]'] > Deff_c)]
num_lc = len(s[s['Effective Diameter [m]'] > 10.])
nsample = int(200/num_lc*len(s))
print(nsample, num_lc, len(s))

scores = {}
errors = {}
samples = []
for j in range(20):
    samples.append((j+1)*100)
    nsample = int((j+1)*100/num_lc*len(s))
    total_score = {}
    for i in range(30):
        scs = s.sample(nsample, replace=False)
        model_stats = gt.score_model(synthetics=scs, Deff_c=5.65, N_c=5, 
                                     size=10, nbins=[15, 10, 15, 15, 10, 15, 10, 10])
        for stat in model_stats:
            try:
                total_score[stat].append(model_stats[stat])
            except KeyError:
                total_score[stat] = [model_stats[stat]]
    for stat in model_stats:
        try:
            scores[stat].append(np.mean(total_score[stat]))
            errors[stat].append(np.std(total_score[stat]))
        except KeyError:
            scores[stat] = [np.mean(total_score[stat])]
            errors[stat] = [np.std(total_score[stat])]


In [ ]:
plt.figure()
model_stats =  gt.score_model(synthetics=s, Deff_c=5.65, N_c=5, size=10, nbins=[15, 10, 15, 15, 10, 15, 10, 10])
for stat in scores:
    if stat == 'No. of Clusters' or stat == 'C-S ratio':
        break
    plt.errorbar(samples, np.array(scores[stat])/model_stats[stat], 
                 np.array(errors[stat])/model_stats[stat], fmt='o', ls='', label=stat)
    plt.xlabel('Number of samples')
    plt.ylabel('Chi-square statistic')
    plt.xlim(0., 2100.)
    plt.legend()

In [ ]:
model_stats =  gt.score_model(synthetics=s, Deff_c=5.65, N_c=5, size=10, nbins=[15, 10, 15, 15, 10, 15, 10, 10])
plt.figure(figsize=(5,2.5))
stat = 'Total'
plt.subplot(1,2,1)
plt.errorbar(samples, scores[stat], errors[stat], fmt='o', ls='', ms=5)
line = [model_stats[stat]]*len(samples)
plt.plot(samples, line, 'k:')
plt.xlabel('Number of samples', fontsize=8)
plt.ylabel('Chi-square statistic', fontsize=8)
plt.xlim(0., 2100.)
plt.ylim(2.,12.)
plt.tick_params(labelsize=8)
plt.annotate('(a)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

stat = 'C-S ratio'
plt.subplot(1,2,2)
plt.errorbar(samples, scores[stat], errors[stat], fmt='o', ls='', ms=5)
line = [model_stats[stat]]*len(samples)
plt.plot(samples, line, 'k:')
plt.xlabel('Number of samples', fontsize=8)
plt.ylabel('Cluster-Singular ratio', fontsize=8)
plt.xlim(0., 2100.)
plt.tick_params(labelsize=8)
plt.annotate('(b)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

plt.tight_layout()

plt.savefig('./Figures/Figure6.pdf')

## Effect of model parameters

In [ ]:
sf = pd.read_csv('./model-data/statistics.csv', index_col='ID')
r = pd.read_csv('./model-data/record.csv', index_col='ID')
aa = sf.join(r)
aa.drop(['Atmosphere', 'Crater rad. (min)', 'Mass SFD exp.', 'Samples',
       'Max. density', 'Min. density', 'Min. mass', 'Drag coef.', 'No. of Clusters'], axis=1, inplace=True)
aa.columns

In [ ]:
runs = [201,202,203,204,205]
a = aa[aa.index.isin(runs)]
a['error'] = errors['C-S ratio'][1]*a['C-S ratio']
plt.figure(figsize=(5,2.5))
plt.subplot(1,2,1)
plt.errorbar(a['Med. strength'], a['C-S ratio'], a['error'], fmt='o', ls='', ms=5, label="w = 1.25") #, c=a['Var. strength'])

runs = [207,231,235]
a = aa[aa.index.isin(runs)]
a['error'] = errors['C-S ratio'][1]*a['C-S ratio']
plt.errorbar(a['Med. strength'], a['C-S ratio'], a['error'], fmt='o', ls='', ms=5, label="w = 0.75") #, c=a['Var. strength'])

runs = [208,232,236]
a = aa[aa.index.isin(runs)]
a['error'] = errors['C-S ratio'][1]*a['C-S ratio']
plt.errorbar(a['Med. strength'], a['C-S ratio'], a['error'], fmt='o', ls='', ms=5, label="w = 1.") #, c=a['Var. strength'])

runs = [209,233,237]
a = aa[aa.index.isin(runs)]
a['error'] = errors['C-S ratio'][1]*a['C-S ratio']
plt.errorbar(a['Med. strength'], a['C-S ratio'], a['error'], fmt='o', ls='', ms=5, label="w = 1.5") #, c=a['Var. strength'])

#plt.colorbar()
plt.yscale('log')
plt.xscale('log')
plt.xlim(80.,1200.)
plt.ylim(0.5,50.)
plt.legend(fontsize=7)
plt.xlabel('Median strength, $Y_{med}$ [kPa]', fontsize=8)
plt.ylabel('Cluster-Singular ratio', fontsize=8)
x = np.logspace(np.log10(80),np.log10(1200))
plt.tick_params(labelsize=8)
plt.annotate('(a)', xy=(0.05, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
plt.fill_between(x, (x*0+1.45-0.3), (x*0+1.45+0.3), alpha = 0.2)

runs = [201,206,207,208,209]
a = aa[aa.index.isin(runs)]
a['error'] = errors['C-S ratio'][1]*a['C-S ratio']
plt.subplot(1,2,2)
plt.errorbar(a['Var. strength'], a['C-S ratio'], a['error'], fmt='o', ls='', ms=5, label="330 kPa") #, c=a['Var. strength'])

runs = [204,230,231,232,233]
a = aa[aa.index.isin(runs)]
a['error'] = errors['C-S ratio'][1]*a['C-S ratio']
plt.errorbar(a['Var. strength'], a['C-S ratio'], a['error'], fmt='o', ls='', ms=5, label="500 kPa") #, c=a['Var. strength'])

runs = [203,234,235,236,237]
a = aa[aa.index.isin(runs)]
a['error'] = errors['C-S ratio'][1]*a['C-S ratio']
plt.errorbar(a['Var. strength'], a['C-S ratio'], a['error'], fmt='o', ls='', ms=5, label="200 kPa") #, c=a['Var. strength'])

#plt.colorbar()
plt.yscale('log')
#plt.xscale('log')
plt.xlim(0.25,1.75)
plt.ylim(0.5,50.)
plt.xlabel('Strength range, w', fontsize=8)
plt.ylabel('Cluster-Singular ratio', fontsize=8)
x = np.linspace(0,2)
leg = plt.legend(fontsize=7,title='$Y_{med}$')
leg.get_title().set_fontsize('7')
plt.fill_between(x, (x*0+1.45-0.3), (x*0+1.45+0.3), alpha = 0.2)
plt.annotate('(b)', xy=(0.05, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
plt.tick_params(labelsize=8)
plt.tight_layout()
plt.savefig('./Figures/Figure7.pdf')

In [ ]:
def plot_cdf_parameter(oc, scs, labels, colors, char, ax, N_c=5, Deff_c=5.65):

    # Plot observations
    ocdf = oc[char].value_counts().sort_index()[::-1].cumsum()/len(oc[char])
    ocdf.plot(label='Obs.',color='Black', linewidth=4, alpha=0.5, ax=ax)

    # Plot model results
    for s, llabel, lcolor in zip(scs, labels, colors):
        sc, ss, sb = gt.classify_clusters(s)
        sc = sc[(sc['No. of Craters'] > N_c) & (sc['Effective Diameter [m]'] > Deff_c)]
        cdf = sc[char].value_counts().sort_index()[::-1].cumsum()/len(sc[char])
        cdf.plot(label=llabel, linewidth=2, ax=ax, color=lcolor)
      
    # Labels, limits, etc.
    ax.set_xlabel(char, fontsize=8)
    if (char != "CSFD exponent" and char != "Aspect Ratio" and 
        char != "Large Crater Fraction"):
        ax.set_xscale('log')
        ax.set_ylim(0., 1.E0)
    else:
        ax.set_ylim(0., 1.)
    ax.legend(fontsize=6, loc=3)
    ax.tick_params(labelsize=8)
    ax.set_ylabel('Cumulative Frequency', fontsize=8)

In [ ]:
runs = [201,214,215,216,217]
a = aa[aa.index.isin(runs)]
a['error'] = errors['Aspect Ratio'][1]*a['Aspect Ratio']
plt.figure(figsize=(5.,7.5))
plt.subplot(3,2,1)
plt.errorbar(a['Lift coef.'], a['Aspect Ratio'], a['error'], fmt='o', ls='')
plt.yscale('log')
plt.xlim(0.005,0.035)
plt.ylim(0.5,30.)
plt.xlabel('Lift coef.', fontsize=8)
plt.ylabel('Aspect Ratio Chi-square statistic', fontsize=8)
plt.tick_params(labelsize=8)
plt.annotate('(a)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

ax = plt.subplot(3,2,2)
ss = [pd.read_csv("./model-data/output-214.csv"), 
      pd.read_csv("./model-data/output-215.csv"), 
      pd.read_csv("./model-data/output-216.csv")]
plot_cdf_parameter(observedclusters, ss, 
                   labels=['$C_L=0.01$','$C_L=0.015$','$C_L=0.025$'], 
                   colors=['b','k','r'], char='Aspect Ratio', ax=ax)
plt.annotate('(b)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)

runs = [201,210,211,212,213]
a = aa[aa.index.isin(runs)]
a['error'] = errors['Dispersion [m]'][1]*a['Dispersion [m]']
plt.subplot(3,2,3)
plt.errorbar(a['Frag sep coef (min)'], a['Dispersion [m]'], a['error'], fmt='o', ls='') #, c=a['Var. strength'])
#plt.colorbar()
plt.yscale('log')
#plt.xscale('log')
#plt.xlim(0.25,1.75)
plt.ylim(0.5,30.)
plt.xlabel('Frag. separation coef.', fontsize=8)
plt.ylabel('Dispersion Chi-square statistic', fontsize=8)
plt.annotate('(c)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)
plt.tick_params(labelsize=8)

ax = plt.subplot(3,2,4)
ss = [pd.read_csv("./model-data/output-210.csv"), 
      pd.read_csv("./model-data/output-201.csv"), 
      pd.read_csv("./model-data/output-213.csv")]
plot_cdf_parameter(observedclusters, ss, 
                   labels=['$C_S=0.1$','$C_S=1.1$','$C_S=2.1$'], 
                   colors=['b','k','r'], char='Dispersion [m]', ax=ax)
plt.annotate('(d)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)
ax.set_xlim([3.E0, 3.E3])

runs = [201,218,219,220,221]
a = aa[aa.index.isin(runs)]
a['error'] = errors['CSFD exponent'][1]*a['CSFD exponent']
plt.subplot(3,2,5)
plt.errorbar(a['Strength scaling disp.'], a['CSFD exponent'], a['error'], fmt='o', ls='') #, c=a['Var. strength'])
#plt.colorbar()
plt.yscale('log')
#plt.xscale('log')
#plt.xlim(0.25,1.75)
plt.ylim(0.1,100.)
plt.xlabel('Strength scaling variance', fontsize=8)
plt.ylabel('CSFD exponent Chi-square statistic', fontsize=8)
plt.annotate('(e)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)
plt.tick_params(labelsize=8)

ax = plt.subplot(3,2,6)
ss = [pd.read_csv("./model-data/output-218.csv"), 
      pd.read_csv("./model-data/output-201.csv"), 
      pd.read_csv("./model-data/output-221.csv")]
plot_cdf_parameter(observedclusters, ss, 
                   labels=['$\delta=0.$','$\delta=0.5$','$\delta=1.0$'], 
                   colors=['b','k','r'], char='CSFD exponent', ax=ax)
plt.annotate('(f)', xy=(0.95, 0.9), ha='right', xycoords='axes fraction', fontsize=10)
ax.set_xlim([-4, 0.])

plt.tight_layout()
plt.savefig('./Figures/Figure8.pdf')

## Impact angle on aspect ratio

In [ ]:
ID = "198"
Deff_c = 5.65; N_c = 5
synthetics=pd.read_csv('./model-data/output-'+ID+'.csv')
sc, ss, sb = gt.classify_clusters(synthetics)
sc.sort_values(by='No. of Craters', inplace=True)
sc = sc[sc['No. of Craters'] > N_c]

aa = np.arange(0.,91,1.)
ar = np.sin(np.radians(aa))

plt.figure(figsize=(5,3))
plt.scatter(sc['Angle'], sc['Aspect Ratio'], c=np.log10(sc['No. of Craters']), cmap='Blues', s=12)
cbar = plt.colorbar(ticks=[1,2,3], shrink=0.8)
cbar.ax.tick_params(labelsize=8)
cbar.ax.set_ylabel('$\log_{10}$(No. of Craters)', fontsize=8)
plt.plot(aa, ar, 'k:', lw=1)
plt.xlabel('Impact angle [$^\circ$]', fontsize=8)
plt.ylabel('Aspect Ratio', fontsize=8)
plt.tick_params(labelsize=8)
plt.tight_layout()
plt.savefig('./Figures/Figure9.pdf')

## Input distributions by outcome

In [ ]:
sns.set_context("paper", rc={"axes.labelsize":12}) 
ID = "198"
Deff_c = 5.65; N_c = 5
synthetics=pd.read_csv('./model-data/output-'+ID+'.csv')

fig = plt.figure(figsize=(15,8))
fig.add_subplot(231)
synthetics['Type'] = ''
synthetics['Type'][synthetics['No. of Craters'] > 1] = 'Clusters'
synthetics['Type'][synthetics['No. of Craters'] == 1] = 'Craters'
synthetics['Type'][synthetics['No. of Craters'] == 0] = 'Airbursts'
synthetics['Radius [m]'] = synthetics['Radius [m]'] * 100.
synthetics['Ablation coef.'] = synthetics['Ablation coef.'] * 1.E9

var = 'Velocity [km/s]'
gt.comp_hist(synthetics, var, bin_list=15, log_scale=False)
plt.title('(a)', fontsize=12)
plt.tick_params(labelsize=12)
plt.xlabel(var, fontsize=12)

fig.add_subplot(232)
var = 'Density [kg/m3]'
gt.comp_hist(synthetics, var, bin_list=15, log_scale=False)
plt.title('(b)', fontsize=12)
plt.tick_params(labelsize=12)
plt.xlabel(var, fontsize=12)

fig.add_subplot(233)
#var = 'Radius [m]'
var = 'Ablation coef.'
gt.comp_hist(synthetics, var, bin_list=15, log_scale=False)
plt.title('(c)', fontsize=12)
plt.tick_params(labelsize=12)
plt.xlabel(var+' [$\\times 10^{-9}$ m$^2$/J]', fontsize=12)
#plt.xlabel('Radius [cm]')
#plt.yscale('log')
#plt.ylim(1.E-4,1.)

fig.add_subplot(234)
var = 'Strength [kPa]'
gt.comp_hist(synthetics, var, bin_list=15, log_scale=True)
plt.title('(d)', fontsize=12)
plt.tick_params(labelsize=12)
plt.xlabel(var, fontsize=12)

fig.add_subplot(235)
var = 'Mass [kg]'
gt.comp_hist(synthetics, var, bin_list=15, log_scale=True)
plt.title('(e)', fontsize=12)
plt.tick_params(labelsize=12)
plt.xlabel(var, fontsize=12)
plt.yscale('log')
plt.ylim(1.E-4,1.)

fig.add_subplot(236)
var = 'Angle'
gt.comp_hist(synthetics, var, bin_list=15, log_scale=False)
plt.title('(f)', fontsize=12)
plt.tick_params(labelsize=12)
plt.xlabel(var, fontsize=12)

plt.tight_layout()
plt.savefig('./Figures/Figure10.pdf')

## Estimate of impact flux

In [ ]:
# Compare SFDs of Clusters and Singles
char = "Effective Diameter [m]"
years = 3.467     # Adjust this time to match observed CFD with Daubar 2013/2014 rate
area = 144.8E6   # Area of Mars
tao = years*area # For scaling observed rates

# Average of two estimates
ME_ratio = 1. #0.5*(3.22/1.58+2.04/1.58)
f_asteroid = 0.4882 #0.67

# Observations
os = pd.read_csv('./obs-data/observedsingulars.csv')
oc = pd.read_csv('./obs-data/observedclusters.csv')
oall = pd.concat([os, oc], sort=False)

# Daubar rates
# Cumulative rate from Crater stats
oi = pd.read_csv('./obs-data/ctx_ctx_update_140418.stat', header=None, delimiter='\s+', comment='#', 
                 usecols=[0,5,6], names=['D', 'N', 'N_err'])

# Use list of crater diameters to get N(D>10 m)
area_oid = 46755681.00
oid = pd.read_csv('./obs-data/ctx_ctx_update_140418.diam', header=None, delimiter='\s+', comment='#', skiprows=10, names=['D'])
Ng10_obs = oid.D[oid.D > 0.01].count()/area_oid
print("Observed N(D>10 m) = ", Ng10_obs)

# Simulation
s = pd.concat([pd.read_csv('./model-data/output-196.csv'), 
               pd.read_csv('./model-data/output-197.csv'),
               pd.read_csv('./model-data/output-198.csv')])

# Time for scaling simulated rates
ta = area*gt.time_on_mars(15., len(s), ME_ratio=ME_ratio) / f_asteroid

sc, ss, sb = gt.classify_clusters(s)
sall = pd.concat([ss, sc], sort=False)

# No fragmentation and no atmosphere models. . . 
no_atmos= pd.read_csv('./model-data/output-257.csv').sample(len(sall), replace=True)

# Compare numbers
print('Singles (model vs obs.):  ', len(ss[char]), len(os[char]))
print('Clusters (model vs obs.): ', len(sc[char]), len(oc[char]))

fig = plt.figure(figsize=(7.5,3.75))
from matplotlib import gridspec
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1.5]) 
ax0 = plt.subplot(gs[0])
ocdf = os[char].value_counts().sort_index()[::-1].cumsum()/tao
ocdf.plot(label='Obs. Singles',color='Black', linewidth=4, alpha=0.4)
ocdf = oc[char].value_counts().sort_index()[::-1].cumsum()/tao
ocdf.plot(label='Obs. Clusters',color='Red', linewidth=4, alpha=0.4)
ocdf = oall[char].value_counts().sort_index()[::-1].cumsum()/tao
ocdf.plot(label='Obs. Combined', color='Blue', linewidth=4, alpha=0.4)

# No atmosphere model
cdf = no_atmos[char].value_counts().sort_index()[::-1].cumsum()/ta
cdf.plot(label='No atmosphere', color='Green', linewidth=1, linestyle='dashed')

# Best-fit model
cdf = ss[char].value_counts().sort_index()[::-1].cumsum()/ta
cdf.plot(label='Model Singles', color='Black', linewidth=2)
if len(sc) > 0:
    cdf = sc[char].value_counts().sort_index()[::-1].cumsum()/ta
    cdf.plot(label='Model Clusters', color='Red', linewidth=2)
cdf = sall[char].value_counts().sort_index()[::-1].cumsum()/ta
cdf.plot(label='Model Combined', color='Blue', linewidth=2)

plt.errorbar(oi.D*1000, oi.N, oi.N_err, fmt='ko', ls='', label='Daubar et al. (2014)')

plt.legend(loc=0, fontsize=6)
plt.xscale("log"); plt.yscale("log")
plt.xlabel(char, fontsize=8); plt.ylabel("Cumulative Frequency per sq. km per year", fontsize=8)
plt.xlim(1.,100.); plt.ylim(6.E-9, 3.E-5)
plt.tick_params(labelsize=8)
plt.grid()
plt.annotate('(a)', xy=(0.05, 0.925), ha='left', xycoords='axes fraction', fontsize=10)

# Find the N(D>10 m) rate in the Neidhardt observations and simulations
N_obs_10 = ocdf[ocdf.index > 10.].iloc[-1]
print("New observed N(D>10 m) = ", N_obs_10)
N_model_10 = cdf[cdf.index > 10.].iloc[-1]
print("Simulated N(D>10 m) = ", N_model_10)

# Plot simulated cratering rate scaled in RME-fA space different
ax1 = plt.subplot(gs[1])
fA = np.linspace(0.,1.,101)
RME = np.linspace(0.5,3.5,101)
# N_model_10/(f_asteroid*ME_Ratio) is the simulated N(>10 m) rate 
# if all meteoroids are asteroids and the flux per unit area on Earth and Mars are the same
# Hence, the ratio N_obs_10*(f_asteroid*ME_ratio/N_model_10) measures how much lower the observed
# rate is than the simulation prediction under those assumptions. We can use this to specify
# the combination of RME and fA that are required to achieve the same result. . .
RME = N_obs_10*(f_asteroid*ME_ratio/N_model_10)/fA

# Plot the RME-fA relationships for different N(>10 m) rates relative to the observed one
plt.plot(fA,0.5*RME,'k-')
plt.plot(fA,RME, 'k-', lw=2)
plt.plot(fA,1.5*RME, 'k-')
plt.plot(fA,2*RME, 'k-')
plt.plot(fA,3*RME, 'k-')
plt.fill_between([0.,1.],1.291, 2.03, color='cyan', alpha=0.25)
plt.fill_betweenx([0.5,2.5],.65, .8, color='green', alpha=0.25)
plt.xlim(0,1)
plt.tick_params(labelsize=8)
plt.xlabel('Fraction of impact flux that is asteroidal $f_A$', fontsize=8)
plt.ylim(1.,2.5)
plt.ylabel('Mars/Earth impact flux ratio $R_\mathrm{ME}$', fontsize=8)
plt.annotate('(b)', xy=(0.95, 0.925), ha='right', xycoords='axes fraction', fontsize=10)
plt.text(0.24,2.24, '$N_{>10} = 4.06\\times10^{-7}$',
        ha='center', va='center', rotation=-81, fontsize=7)
plt.text(0.13,2.24, '$N_{>10} = 2.03\\times10^{-7}$',
        ha='center', va='center', rotation=-85, fontsize=7)
plt.text(0.35,2.24, '$N_{>10} = 6.09\\times10^{-7}$',
        ha='center', va='center', rotation=-76, fontsize=7)
plt.text(0.46,2.24, '$N_{>10} = 8.12\\times10^{-7}$',
        ha='center', va='center', rotation=-72, fontsize=7)
plt.text(0.68,2.24, '$N_{>10} = 1.62\\times10^{-6}$',
        ha='center', va='center', rotation=-62, fontsize=7)

plt.tight_layout()
plt.savefig('./Figures/Figure11.pdf')

## Analysis of fragment properties at impact

In [ ]:
synthetics=pd.read_csv('./model-data/output-198.csv', index_col=0) 
fragments =pd.read_csv('./model-data/frag_summary-198.csv', index_col=0)
output = synthetics.join(fragments)
Dec = 2.
scf = output[(output['No. of Craters'] > 1) & (output['Effective Diameter [m]'] > Dec)].filter(items=['Effective Diameter [m]', 'No. of Craters', 'No. of fragments',
                    'Strength [kPa]', 'Velocity [km/s]', 'Mass [kg]', 'Angle', 
                    'Mean angle', 'Mean mass [kg]', 'Mean momentum [Ns]', 
                    'Mean time [s]', 'Mean velocity [km/s]', 'Mean vert. momentum [Ns]',
                    'Median angle', 'Median mass [kg]', 'Median momentum [Ns]', 
                    'Median time [s]', 'Median velocity [km/s]', 'Median vert. momentum [Ns]',
                    'Maximum angle', 'Maximum mass [kg]', 'Maximum momentum [Ns]', 
                    'Maximum time [s]', 'Maximum velocity [km/s]', 'Maximum vert. momentum [Ns]',
                    'Total mass [kg]', 'Total momentum [Ns]', 'Total vert. momentum [Ns]'])
ssf = output[(output['No. of Craters'] == 1) & (output['Effective Diameter [m]'] > Dec)].filter(items=['Effective Diameter [m]', 'No. of Craters', 'No. of fragments',
                    'Strength [kPa]', 'Velocity [km/s]', 'Mass [kg]', 'Angle', 
                    'Mean angle', 'Mean mass [kg]', 'Mean momentum [Ns]', 
                    'Mean time [s]', 'Mean velocity [km/s]', 'Mean vert. momentum [Ns]',
                    'Median angle', 'Median mass [kg]', 'Median momentum [Ns]', 
                    'Median time [s]', 'Median velocity [km/s]', 'Median vert. momentum [Ns]',
                    'Maximum angle', 'Maximum mass [kg]', 'Maximum momentum [Ns]', 
                    'Maximum time [s]', 'Maximum velocity [km/s]', 'Maximum vert. momentum [Ns]',
                    'Total mass [kg]', 'Total momentum [Ns]', 'Total vert. momentum [Ns]'])

In [ ]:
plt.figure(figsize=(5,3))
xvar = 'Effective Diameter [m]'
yvar = 'Total vert. momentum [Ns]'
plt.subplot(1,2,1)
plt.scatter(ssf[xvar], ssf[yvar], c='k', alpha=0.1, label='Singulars', s=8)
plt.scatter(scf[xvar], scf[yvar], c='r', alpha=0.1, label='Clusters', s=8)
plt.xscale('log')
plt.yscale('log')
plt.xlabel(xvar, fontsize=8)
plt.ylabel(yvar, fontsize=8)
plt.legend(fontsize=7)
plt.tick_params(labelsize=8)
plt.annotate('(a)', xy=(-0.3, 0.925), ha='left', xycoords='axes fraction', fontsize=10)

plt.subplot(1,2,2)

sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8}) 
sns.kdeplot(x=scf['Effective Diameter [m]'], y=scf['Median time [s]'], 
            log_scale=True, cmap='light:b', shade=True)
plt.ylim(5E-4,50.)
plt.tick_params(labelsize=8)
plt.xlabel('Effective Diameter [m]', fontsize=8)
plt.ylabel('Median time [s]', fontsize=8)
plt.annotate('(b)', xy=(-0.3, 0.925), ha='left', xycoords='axes fraction', fontsize=10)
plt.tight_layout()
plt.savefig('./Figures/Figure12.pdf')

In [ ]:
# Fudge to move and change kdeplot legend 
# https://github.com/mwaskom/seaborn/issues/2280#issuecomment-692350136
def move_legend(ax, new_loc, **kws):
    old_legend = ax.legend_
    handles = old_legend.legendHandles
    labels = [t.get_text() for t in old_legend.get_texts()]
    title = old_legend.get_title().get_text()
    ax.legend(handles, labels, loc=new_loc, **kws)

scf['Type'] = 'Clusters'
ssf['Type'] = 'Singulars'
sf = pd.concat([scf, ssf])
sf['Deceleration factor (median)'] = sf['Median velocity [km/s]']/sf['Velocity [km/s]']
sf['Deceleration factor (maximum)'] = sf['Maximum velocity [km/s]']/sf['Velocity [km/s]']
sf['Mass ratio'] = sf['Total mass [kg]']/sf['Mass [kg]']

plt.figure(figsize=(3.75,7.5))
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8}) 
xvar = 'Effective Diameter [m]'
yvar = 'Deceleration factor (maximum)'
plt.subplot(3,1,1)
plt.annotate('(a)', xy=(0.025, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
plt.ylim(0.1,1.2)
sns.kdeplot(data=sf, x=xvar, y=yvar, 
            log_scale=True, hue='Type', legend=False)
#plt.xscale('log')
plt.xlim(1.,100.)
plt.subplot(3,1,2)
plt.annotate('(b)', xy=(0.025, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
yvar = 'Deceleration factor (median)'
sns.kdeplot(data=sf, x=xvar, y=yvar, 
            log_scale=True, hue='Type', legend=False)
plt.ylim(0.1,1.2)
#plt.xscale('log')
plt.xlim(1.,100.)

ax = plt.subplot(3,1,3)
plt.annotate('(c)', xy=(0.025, 0.9), ha='left', xycoords='axes fraction', fontsize=10)
yvar = 'Mass ratio'
t = sns.kdeplot(data=sf, x=xvar, y=yvar, 
            log_scale=True, hue='Type', legend=True)
move_legend(t, "lower right", fontsize=7)
plt.ylim(0.01,1.2)
#plt.xscale('log')
plt.xlim(1.,100.)
plt.tight_layout()
plt.savefig('./Figures/Figure13.pdf')